# Importing libraries

In [1]:
import pandas as pd
import numpy as np
import math
import os
import sys
import json
import networkx as nx
from z3 import *

# Current and parent directories

In [2]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)

# Read zone information

In [3]:
zones = pd.read_excel(str(parent_directory) + '/data/Aras-Information.xlsx', sheet_name='Zone-Info')
num_zones = len(zones)

# Parameters

In [4]:
num_timeslots = 1440
num_zones = 5

# Attack schedule

In [5]:
def attack_scheduling(list_time_min, list_time_max, start_time, final_time):
    prev_stay = 1
    prev_schedule = -1
    ultimate_cost = 0
    final_schedule = schedule = np.zeros((final_time - start_time))
    num_timeslots = interval = 10
    
    
    for init_time in range(start_time, final_time, interval):
            
        cost = [Int( 'cost_' + str(i)) for i in range(num_zones)]
        zones = [Int( 'zones_' + str(i)) for i in range(num_zones)]

        schedule = [Int( 'schedule_' + str(i)) for i in range(init_time, init_time + interval)]
        stay = [Int( 'stay_' + str(i)) for i in range(interval)]
        slot_cost = [Int( 'slot_cost_' + str(i)) for i in range(interval)]

        total_cost = Int('total_cost')

        o = Optimize()
        o.add(cost[0] == 0)
        o.add(cost[1] == 1)
        o.add(cost[2] == 2)
        o.add(cost[3] == 4)
        o.add(cost[4] == 3)


        ############################################################################
        ################## schedule should be withing a valid zone #################
        ############################################################################
        for t in range(len(schedule)):
            or_constraints = []
            for z in range(num_zones):
                or_constraints.append(schedule[t] == z)
            o.add(Or(or_constraints))

        ###############################################################################################################
        ################## if zone stay threshdold in current time is 0, do not schedule to that zone #################
        ###############################################################################################################

        # base case
        for z in range(num_zones):
            if list_time_min[z][init_time] == []:
                o.add(Implies(schedule[0] != prev_schedule, schedule[0] != z))

        for t in range(1, len(schedule)):
            for z in range(num_zones):
                if list_time_min[z][init_time + t] == []:
                    o.add(Implies(schedule[t] != schedule[t - 1], schedule[t] != z))

        #######################################################################################################################
        ############################################ constraints of stay ######################################################
        #######################################################################################################################
        ######## base case for time 0 ############
        if init_time == 0:
            o.add(stay[0] == 1)
        else:
            o.add(Implies(schedule[0] == prev_schedule, stay[0] == prev_stay + 1))
            o.add(Implies(Not(schedule[0] == prev_schedule), stay[0] == 1))

        for t in range(1, len(schedule)):
            continue_staying = (schedule[t] == schedule[t - 1])
            increment_stay = (stay[t] == stay[t - 1] + 1)
            reset_stay = (stay[t] == 1)

            o.add(Implies(continue_staying, increment_stay))
            o.add(Implies(Not(continue_staying), reset_stay))

        #######################################################################################################################
        ############ move to a zone different that previous timeslot if stay > max threshold in previous timeslot #############
        #######################################################################################################################
        ######## base case for time 0 ############
        o.add(Implies(prev_stay == max(list_time_max[prev_schedule][init_time - prev_stay], default=0), schedule[0] != prev_schedule))

        for t in range(1, len(schedule)):
            max_stay_threshold = 0
            for z in range(num_zones):
                for p_t in range(1, init_time + len(schedule)):
                    continue_staying = (schedule[t] == schedule[t - 1])
                    o.add(Implies(And(schedule[t - 1] == z, stay[t - 1] == p_t, p_t == max(list_time_max[z][init_time + t - p_t], default=0)), Not(continue_staying)))

        #######################################################################################################################
        # must stay in the zone same as the previous timeslot if stay < max && stay is in previous timeslot is out of cluster #
        #######################################################################################################################
        ######## base case for time 0 ############
        ranges_stay_constraints = []
        for k in range(len(list_time_min[prev_schedule][init_time - prev_stay])):
            ranges_stay_constraints.append(And(prev_stay >= list_time_min[prev_schedule][init_time - prev_stay][k], prev_stay <= list_time_max[prev_schedule][init_time - prev_stay][k]))            

        if init_time != 0:
            o.add(Implies(Not(Or(ranges_stay_constraints)), schedule[0] == prev_schedule))

        for t in range(1, len(schedule)):
            for z in range(num_zones):
                for p_t in range(1, init_time + t + 1):
                    ranges_stay_constraints = []
                    for k in range(len(list_time_min[z][init_time + t - p_t])):
                        ranges_stay_constraints.append(And(p_t >= list_time_min[z][init_time + t - p_t][k], p_t <= list_time_max[z][init_time + t - p_t][k]))            

                    continue_staying = (schedule[t] == schedule[t - 1])
                    o.add(Implies(And(schedule[t - 1] == z, stay[t - 1] == p_t, Not(Or(ranges_stay_constraints))), continue_staying))

        for t in range(len(schedule)):
            for z in range(num_zones):
                o.add(Implies(schedule[t] == z, slot_cost[t] == cost[z]))

        o.add(total_cost == Sum(slot_cost))

        o.maximize(total_cost)

        o.check()
        
        if o.check() == unsat:
            print('unsat', init_time)
            print(prev_stay, prev_schedule)
            return final_schedule, ultimate_cost
        
        print(init_time, o.model()[total_cost])
        ultimate_cost += int(str(o.model()[total_cost]))


        for t in range(interval): 
            final_schedule[init_time + t] = int(str(o.model()[schedule[t]]))

        prev_schedule = int(str(o.model()[schedule[-1]]))
        prev_stay = int(str(o.model()[stay[-1]]))
        
        
    return final_schedule, ultimate_cost

# Save linearized attack schedule (STRENGTH) for all houses and occupants

In [ ]:
strength_attack_costs_actual = dict()

for adm_algo in ['DBSCAN', 'K-Means']:
    for house_name in ['A', 'B']:
        for occupant_id in ['1', '2']:
            print("Linearized: House: " + str(house_name) + ", Occupant: " + str(occupant_id) + ", ADM Algo: " + str(adm_algo))                  

            filename = str(parent_directory) + '/data/deadlock-elimination/Linearized_' + str(adm_algo) + '_House-' + str(house_name) + '_Occupant-' + str(occupant_id) + '.json'

            with open(filename, "r") as file:
                data = json.load(file)

            rewards = [0, 1, 2, 4, 3]
            list_time_min = data["List-Time-Min"]
            list_time_max = data["List-Time-Max"]

            attack_schedule = attack_scheduling(list_time_min, list_time_max, 0, num_timeslots)[0]

            all_states = []
            arrival_time = 0
            arrival_zone = attack_schedule[0]
            stay_duration = 1

            for i in range(1, 1440):
                if attack_schedule[i] != attack_schedule[i - 1]:
                    all_states.append(str(arrival_time) + '-' + str(arrival_zone) + '-' + str(stay_duration))
                    stay_duration = 1
                    arrival_zone = attack_schedule[i]
                    arrival_time = i
                else:
                    stay_duration += 1

            if stay_duration > 1:
                all_states.append(str(arrival_time) + '-' + str(int(arrival_zone)) + '-' + str(stay_duration))  


            memory = {"All-States" : all_states, "Attack-Schedule" : attack_schedule[0], "Attack-Cost" : attack_schedule[1]}

            output_filename = str(parent_directory) + '/data/attack-schedules/shatter/Linearized_' + str(adm_algo) + '_House-' + str(house_name) + '_Occupant-' + str(occupant_id) + '.json'

            with open(output_filename, "w") as json_file:
                json.dump(memory, json_file)

Linearized: House: A, Occupant: 1, ADM Algo: DBSCAN
0 30
10 30
20 26
30 10
40 10
50 10
60 37
70 24
80 20
90 20
100 20
110 20
120 20
130 20
140 11
150 10
160 10
170 10
180 10
190 10
200 10
210 10
220 10
230 10
240 10
250 10
260 10
270 10
280 10
290 10
300 10
310 10
320 10
330 10
340 10
350 10
360 10
370 10
380 10
390 10
400 10
410 10
420 10
430 10
440 10
450 10
460 10
470 10
480 10
490 10
500 10
510 10
520 10
530 10
540 10
550 10
560 10
570 10
580 10
590 10
600 10
610 10
620 10
630 19
640 20
650 16
660 27
670 17
680 20
690 19
700 10
710 25
720 40
730 40
740 40
750 40
760 22
770 10
780 13
790 20
800 20
810 20
820 20
830 20
840 20
850 20
860 20
870 20
880 20
890 20
900 20
910 20
920 20
930 20
940 38
950 34
960 20
970 26
980 40
990 40
1000 35
1010 40
1020 40
1030 40
1040 32
1050 32
1060 40
1070 40
1080 38
1090 23
1100 20
1110 20
1120 20
1130 20
1140 20
1150 26
1160 27
1170 24
1180 12
1190 10
1200 10
1210 17
1220 20
1230 38
1240 40
1250 40
1260 0
1270 4
1280 20
1290 20
1300 12
1310 10
1320 

Exception ignored in: <function AstRef.__del__ at 0x7f9c1098f310>
Traceback (most recent call last):
  File "/Users/nurimtiazulhaque/opt/anaconda3/lib/python3.9/site-packages/z3/z3.py", line 346, in __del__
    Z3_dec_ref(self.ctx.ref(), self.as_ast())
  File "/Users/nurimtiazulhaque/opt/anaconda3/lib/python3.9/site-packages/z3/z3core.py", line 1528, in Z3_dec_ref
    _elems.Check(a0)
  File "/Users/nurimtiazulhaque/opt/anaconda3/lib/python3.9/site-packages/z3/z3core.py", line 1457, in Check
    err = self.get_error_code(ctx)
KeyboardInterrupt: 


1330 10
1340 10
1350 20
1360 20
1370 32
1380 26
1390 20
1400 20


In [ ]:
attack_schedule[0]

# Save actual attack schedule (STRENGTH) for all houses and occupants

In [ ]:
strength_attack_costs_actual = dict()

for adm_algo in ['DBSCAN', 'K-Means']:
    for house_name in ['A', 'B']:
        for occupant_id in ['1', '2']:
            print("Actual: House: " + str(house_name) + ", Occupant: " + str(occupant_id) + ", ADM Algo: " + str(adm_algo))                  

            filename = str(parent_directory) + '/data/deadlock-elimination/Actual_' + str(adm_algo) + '_House-' + str(house_name) + '_Occupant-' + str(occupant_id) + '.json'

            with open(filename, "r") as file:
                data = json.load(file)

            rewards = [0, 1, 2, 4, 3]
            list_time_min = data["List-Time-Min"]
            list_time_max = data["List-Time-Max"]

            attack_schedule = attack_scheduling(list_time_min, list_time_max, 0, num_timeslots)

            all_states = []
            arrival_time = 0
            arrival_zone = attack_schedule[0]
            stay_duration = 1

            for i in range(1, 1440):
                if attack_schedule[i] != attack_schedule[i - 1]:
                    all_states.append(str(arrival_time) + '-' + str(arrival_zone) + '-' + str(stay_duration))
                    stay_duration = 1
                    arrival_zone = attack_schedule[i]
                    arrival_time = i
                else:
                    stay_duration += 1

            if stay_duration > 1:
                all_states.append(str(arrival_time) + '-' + str(int(arrival_zone)) + '-' + str(stay_duration))  


            memory = {"All-States" : all_states, "Attack-Schedule" : attack_schedule[0], "Attack-Cost" : attack_schedule[1]}

            output_filename = str(parent_directory) + '/data/attack-schedules/shatter/Actual_' + str(adm_algo) + '_House-' + str(house_name) + '_Occupant-' + str(occupant_id) + '.json'

            with open(output_filename, "w") as json_file:
                json.dump(memory, json_file)